**검증 세트**<br>
1. 훈련 세트: 모델 훈련
2. 검증 세트: 모델 평가
3. 매개변수 바꾸고 가장 좋은 게 나올 때까지 반복
4. 테스트 세트: 최종 점검


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import uniform, randint
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [ ]:
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data,target,test_size=0.2, random_state=42)
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state=42)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


**교차 검증**<br>
검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복함<br>
k-폴드 교차 검증: 훈련 세트를 k개의 부분으로 나누어 교차 검증을 수행

In [ ]:
scores = cross_validate(dt, train_input, train_target)    #기본적으로 5-폴드 교차 검증
print(scores)                                             #fit_time: 훈련 시간, socre_time: 검증 시간
print(np.mean(scores['test_score']))

{'fit_time': array([0.00734806, 0.00583553, 0.00602102, 0.00595331, 0.00567055]), 'score_time': array([0.00068092, 0.00062943, 0.00058699, 0.00061202, 0.00057077]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


**분할기**<br>
폴드를 어떻게 나눌지를 결정함

In [ ]:
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


**하이퍼파라미터 튜닝**<br>
매개변수 바꿔가면서 모델 훈련하기<br><br>
**그리드 서치**<br>
하이퍼파라미터 탐색 & 교차 검증을 한 번에 수행

In [ ]:
params = {'min_impurity_decrease':[0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)    #params마다 1번씩 5-폴드 교차 검증 수행, 모델 훈련 개수 = 5*5 #n_jobs: CPU 코어 수 지정
gs.fit(train_input, train_target)
dt = gs.best_estimator_                                                          #gs 객체에서 가장 성능 좋은 모델의 매개변수 조함이 best_estimator_에 저장되어 있음
print(dt.score(train_input, train_target))
print(gs.best_params_)                                                           #그리드 서치로 찾은 최적의 매개변수
print(gs.cv_results_['mean_test_score'])
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [ ]:
params = {'min_impurity_decrease':np.arange(0.0001,0.001,0.0001),
          'max_depth': range(5,20,1),
          'min_samples_split':range(2,100,10)}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)    #params의 조합마다 1번씩 5-폴드 교차 검증 수행, 모델 훈련 개수 = 9*15*10*5
gs.fit(train_input, train_target)
print(gs.best_params_)                                                           #그리드 서치로 찾은 최적의 매개변수
print(gs.cv_results_['mean_test_score'])

KeyboardInterrupt: ignored

**랜덤 서치**<br>
매개변수 값의 목록이 아닌, 매개변수를 샘플링할 수 있는 확률 분호 객체 전달

In [ ]:
rgen = randint(0,10)                                                              #0부터 10 미만의 정수
rgen.rvs(10)                                                                      #랜덤으로 하나 10번 뽑기
np.unique(rgen.rvs(1000), return_counts=True)                                     #0,1,2,...,9 숫자가 1000번의 트라이 중 몇 개 뽑혔는지
ugen = uniform(0,1)                                                               #0부터 1 미만의 실수
ugen.rvs(10)

array([0.28979324, 0.1706331 , 0.3519023 , 0.80787906, 0.72826314,
       0.8880585 , 0.7367044 , 0.08187801, 0.99022918, 0.34844923])

In [ ]:
params = {'min_impurity_decrease':uniform(0.001,0.001),
          'max_depth': randint(20,50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 45, 'min_impurity_decrease': 0.001025419126744095, 'min_samples_leaf': 13, 'min_samples_split': 20}
0.8647301399274451
0.8561538461538462
